<h1 style="text-align: center; font-family: Verdana; font-size: 32px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; font-variant: small-caps; letter-spacing: 3px; color: #74d5dd; background-color: #ffffff;">Human Protein Atlas - Single Cell Classification</h1>
<h2 style="text-align: center; font-family: Verdana; font-size: 18px; font-style: normal; font-weight: bold; text-decoration: underline; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">Showing You How To Create TFRecords for the Original Slide Dataset</h2>
<h5 style="text-align: center; font-family: Verdana; font-size: 12px; font-style: normal; font-weight: bold; text-decoration: None; text-transform: none; letter-spacing: 1px; color: black; background-color: #ffffff;">CREATED BY: DARIEN SCHETTLER</h5>


<h2 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;">TABLE OF CONTENTS</h2>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#imports">0&nbsp;&nbsp;&nbsp;&nbsp;IMPORTS</a></h3>


---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#setup">1&nbsp;&nbsp;&nbsp;&nbsp;SETUP</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#helper_functions">2&nbsp;&nbsp;&nbsp;&nbsp;HELPER FUNCTIONS</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#dataset">3&nbsp;&nbsp;&nbsp;&nbsp;DATASET PREPERATION</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#create_tfrec">4&nbsp;&nbsp;&nbsp;&nbsp;HOW TO CREATE TFRECORDS</a></h3>



<a style="text-align: font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;" id="imports">0&nbsp;&nbsp;IMPORTS</a>

In [ ]:
print("\n... IMPORTS STARTING ...\n")
print("\n\tVERSION INFORMATION")

# Machine Learning and Data Science Imports
import tensorflow_addons as tfa; print(f"\t\t– TENSORFLOW ADDONS VERSION: {tfa.__version__}");
import tensorflow as tf; print(f"\t\t– TENSORFLOW VERSION: {tf.__version__}");
import pandas as pd; pd.options.mode.chained_assignment = None;
import numpy as np; print(f"\t\t– NUMPY VERSION: {np.__version__}");
import scipy; print(f"\t\t– SCIPY VERSION: {scipy.__version__}");

# Built In Imports
from kaggle_datasets import KaggleDatasets
from collections import Counter
from datetime import datetime
import multiprocessing
from glob import glob
import warnings
import requests
import imageio
import IPython
import urllib
import zipfile
import pickle
import random
import shutil
import string
import math
import tqdm
import time
import gzip
import io
import os
import gc
import re

# Visualization Imports
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import plotly.express as px
import seaborn as sns
from PIL import Image
import matplotlib; print(f"\t\t– MATPLOTLIB VERSION: {matplotlib.__version__}");
import plotly
import PIL
import cv2

# PRESETS
LBL_NAMES = ["Nucleoplasm", "Nuclear Membrane", "Nucleoli", "Nucleoli Fibrillar Center", "Nuclear Speckles", "Nuclear Bodies", "Endoplasmic Reticulum", "Golgi Apparatus", "Intermediate Filaments", "Actin Filaments", "Microtubules", "Mitotic Spindle", "Centrosome", "Plasma Membrane", "Mitochondria", "Aggresome", "Cytosol", "Vesicles", "Negative"]
INT_2_STR = {x:LBL_NAMES[x] for x in np.arange(19)}
INT_2_STR_LOWER = {k:v.lower().replace(" ", "_") for k,v in INT_2_STR.items()}
STR_2_INT_LOWER = {v:k for k,v in INT_2_STR_LOWER.items()}
STR_2_INT = {v:k for k,v in INT_2_STR.items()}
FIG_FONT = dict(family="Helvetica, Arial", size=14, color="#7f7f7f")
LABEL_COLORS = [px.colors.label_rgb(px.colors.convert_to_RGB_255(x)) for x in sns.color_palette("Spectral", len(LBL_NAMES))]
LABEL_COL_MAP = {str(i):x for i,x in enumerate(LABEL_COLORS)}

print("\n\n... IMPORTS COMPLETE ...\n")

<a style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;" id="setup">1&nbsp;&nbsp;NOTEBOOK SETUP</a>

In [ ]:
# Define the root data directory
DATA_DIR = "/kaggle/input/hpa-single-cell-image-classification"

# Define the paths to the training and testing tfrecord and image folders respectively
TRAIN_TFREC_DIR = os.path.join(DATA_DIR, "train_tfrecords")
TEST_TFREC_DIR = os.path.join(DATA_DIR, "test_tfrecords")

# Capture all the relevant full tfrec paths
TRAIN_TFREC_PATHS = sorted(tf.io.gfile.glob(os.path.join(TRAIN_TFREC_DIR, '*.tfrec')))
TEST_TFREC_PATHS = sorted(tf.io.gfile.glob(os.path.join(TEST_TFREC_DIR, '*.tfrec')))
print(f"\n... The number of training tfrecord files is {len(TRAIN_TFREC_PATHS)} ...")
print(f"... The number of testing tfrecord files is {len(TEST_TFREC_PATHS)} ...\n")

# Define paths to the relevant csv files
TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")
SS_CSV = os.path.join(DATA_DIR, "sample_submission.csv")

# Create the relevant dataframe objects
train_df = pd.read_csv(TRAIN_CSV)
ss_df = pd.read_csv(SS_CSV)

print("\n\nTRAIN DATAFRAME\n\n")
display(train_df.head(3))

print("\n\nSAMPLE SUBMISSION DATAFRAME\n\n")
display(ss_df.head(3))

<a style="text-align: font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;" id="helper_functions">2&nbsp;&nbsp;HELPER FUNCTIONS</a>

In [ ]:
def load_image_from_channel_paths(channel_paths):
    rgby = [np.asarray(Image.open(path), np.uint8) for path in channel_paths]
    return np.stack(rgby, axis=-1)

    
def convert_rgby_to_rgb(arr):
    """ Convert a 4 channel (RGBY) image to a 3 channel RGB image.
    
    Advice From Competition Host/User: lnhtrang

    For annotation (by experts) and for the model, I guess we agree that individual 
    channels with full range px values are better. 
    In annotation, we toggled the channels. 
    For visualization purpose only, you can try blending the channels. 
    For example, 
        - red = red + yellow
        - green = green + yellow/2
        - blue=blue.
        
    Args:
        arr (numpy array): The RGBY, 4 channel numpy array for a given image
    
    Returns:
        RGB Image
    """
    
    rgb_arr = np.zeros_like(arr[..., :-1])
    rgb_arr[..., 0] = arr[..., 0]
    rgb_arr[..., 1] = arr[..., 1]+arr[..., 3]/2
    rgb_arr[..., 2] = arr[..., 2]
    
    return rgb_arr


def plot_ex(arr, figsize=(20,6), title=None, plot_merged=True, rgb_only=False):
    """ Plot 4 Channels Side by Side """
    if plot_merged and not rgb_only:
        n_images=5 
    elif plot_merged and rgb_only:
        n_images=4
    elif not plot_merged and rgb_only:
        n_images=4
    else:
        n_images=3
    plt.figure(figsize=figsize)
    if type(title) == str:
        plt.suptitle(title, fontsize=20, fontweight="bold")

    for i, c in enumerate(["Red Channel – Microtubles", "Green Channel – Protein of Interest", "Blue - Nucleus", "Yellow – Endoplasmic Reticulum"]):
        if not rgb_only:
            ch_arr = np.zeros_like(arr[..., :-1])        
        else:
            ch_arr = np.zeros_like(arr)
        if c in ["Red Channel – Microtubles", "Green Channel – Protein of Interest", "Blue - Nucleus"]:
            ch_arr[..., i] = arr[..., i]
        else:
            if rgb_only:
                continue
            ch_arr[..., 0] = arr[..., i]
            ch_arr[..., 1] = arr[..., i]
        plt.subplot(1,n_images,i+1)
        plt.title(f"{c.title()}", fontweight="bold")
        plt.imshow(ch_arr)
        plt.axis(False)
        
    if plot_merged:
        plt.subplot(1,n_images,n_images)
        
        if rgb_only:
            plt.title(f"Merged RGB", fontweight="bold")
            plt.imshow(arr)
        else:
            plt.title(f"Merged RGBY into RGB", fontweight="bold")
            plt.imshow(convert_rgby_to_rgb(arr))
        plt.axis(False)
        
    plt.tight_layout(rect=[0, 0.2, 1, 0.97])
    plt.show()
    
    
def flatten_list_of_lists(l_o_l):
    return [item for sublist in l_o_l for item in sublist]


def decode_image(image_data, n_channels=1, resize_to=(512,512), cast_to=tf.uint8):
    image = tf.image.decode_png(image_data, channels=n_channels)    
    image = tf.image.resize(image, resize_to) 
    return tf.cast(image, cast_to)


def str_2_multi_hot_encoding(tfstring, n_classes=19):
    ragged_indices = tf.strings.to_number(tf.strings.split(tfstring, sep="|"), out_type=tf.int32)
    one_hot_stack = tf.one_hot(ragged_indices, depth=n_classes)
    return tf.reduce_max(one_hot_stack, axis=-2)


def decode(serialized_example, multihot=False, n_channels=1, resize_to=(512,512)):
    """ Parses a set of features and label from the given `serialized_example`.
        
        It is used as a map function for `dataset.map`

    Args:
        serialized_example (tf.Example): A serialized example
        is_test (bool, optional): Whether to allow for the label feature
        
    Returns:
        A decoded tf.data.Dataset object representing the tfrecord dataset
    """
    # Defaults are not specified since both keys are required.
    feature_dict = {
        'image': tf.io.FixedLenFeature(shape=(), dtype=tf.string),
        'image_name': tf.io.FixedLenFeature(shape=(), dtype=tf.string),
        'target': tf.io.FixedLenFeature(shape=(), dtype=tf.string),
    }
    # Define a parser
    features = tf.io.parse_single_example(serialized_example, features=feature_dict)   
    image = decode_image(features['image'], n_channels, resize_to)
    image_name = features["image_name"]
    if multihot:
        label = str_2_multi_hot_encoding(features["target"])
    else:
        label = features["target"]
    return image, image_name, label


def preprocess_tfrec_ds(red, green, blue, yellow, drop_yellow=True, return_id=True):
    (ri, rn, rl), (gi, gn, gl), (bi, bn, bl), (yi, yn, yl) = red, green, blue, yellow
    if drop_yellow:
        combo_img = tf.stack([ri[..., 0], gi[..., 0], bi[..., 0]], axis=-1)
    else:
        combo_img = tf.stack([ri[..., 0], gi[..., 0], bi[..., 0], yi[..., 0]], axis=-1)
    
    if return_id:
        img_id = tf.strings.substr(rn, pos=0, len=36) # 36 is length of id (always)
        return combo_img, img_id, rl
    else:
        return combo_img, rl

<a style="text-align: font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;" id="dataset">3&nbsp;&nbsp;DATASET PREPERATION</a>

In [ ]:
# num_parallel_reads=None forces the order to be preserved
raw_train_ds = tf.data.TFRecordDataset(TRAIN_TFREC_PATHS, num_parallel_reads=None)

# See an example
for raw in raw_train_ds.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw.numpy())
    for k,v in example.features.feature.items():
        print(k)
        if k!="image":
            print(f"\t--> {v.bytes_list.value[0]}")
        else:
            print(f"\t-->{str(v.bytes_list.value[0][:25])+' ... '}")

In [ ]:
train_ds = raw_train_ds.map(decode)

# See examples
print("\n ... NOTICE THE IMAGES ARE CLUMPED TOGETHER BY CHANNEL FOR A GIVEN ID ...\n")
for i, (img, image_name, lbl) in enumerate(train_ds.take(4)):
    print(f"IMAGE SHAPE : {img.shape}")
    print(f"IMG #{i//4} -- IMAGE NAME  : {image_name.numpy().decode()}")
    print(f"IMAGE LABEL : {lbl}\n")

In [ ]:
#####################################################
# ANNOYINGLY THIS DOES NOT WORK AS THE QUADRUPLE OF #
# CHANNEL IMAGES IS IN A DIFFERENT ORDER EVERY TIME #
#####################################################
# red_train_ds = train_ds.shard(4, index=3)
# green_train_ds = train_ds.shard(4, index=2)
# blue_train_ds = train_ds.shard(4, index=0)
# yellow_train_ds = train_ds.shard(4, index=1)
#####################################################

red_train_ds = train_ds.filter(lambda x,y,z: tf.strings.regex_full_match(y, ".*red.*"))
green_train_ds = train_ds.filter(lambda x,y,z: tf.strings.regex_full_match(y, ".*green.*"))
blue_train_ds = train_ds.filter(lambda x,y,z: tf.strings.regex_full_match(y, ".*blue.*"))
yellow_train_ds = train_ds.filter(lambda x,y,z: tf.strings.regex_full_match(y, ".*yellow.*"))


print("\n ... NOTICE THE IMAGES ARE NOW IN THEIR OWN DATASET BY COLOR ...\n")
for (img_r, image_name_r, lbl_r), (img_b, image_name_b, lbl_b) in zip(red_train_ds.take(4), blue_train_ds.take(4)):
    print(f"IMAGE SHAPE : R={img_r.shape} - B={img_b.shape}")
    print(f"IMAGE NAME  : R={image_name_r.numpy().decode()} - B={image_name_b.numpy().decode()}")
    print(f"IMAGE LABEL : R={lbl_r.numpy().decode()} - B={lbl_b.numpy().decode()}")
    print()

In [ ]:
train_ds = tf.data.Dataset.zip((red_train_ds, green_train_ds, blue_train_ds, yellow_train_ds)).map(preprocess_tfrec_ds)

print("\n\t\t... TRAIN EXAMPLES ...\n")
for x,y,z in train_ds.take(3):
    plot_ex(x.numpy(), title=f"ID = {y.numpy().decode()}\nLABELS = {', '.join([INT_2_STR[int(lbl)] for lbl in z.numpy().decode().split('|')])}", rgb_only=True)

<a style="text-align: font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;" id="create_tfrec">4&nbsp;&nbsp;HOW TO RECREATE THE TFRECORDS</a>

In [ ]:
def _bytes_feature(value, is_list=False):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    
    if not is_list:
        value = [value]
    
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

def _float_feature(value, is_list=False):
    """Returns a float_list from a float / double."""
        
    if not is_list:
        value = [value]
        
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _int64_feature(value, is_list=False):
    """Returns an int64_list from a bool / enum / int / uint."""
        
    if not is_list:
        value = [value]
        
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def serialize(image, image_name, target):
    """
    Creates a tf.Example message ready to be written to a file from 4 features.

    Args:
        image (TBD): TBD
        image_name (str): TBD
        target (str): | delimited integers
    
    Returns:
        A tf.Example Message ready to be written to file
    """
    
    # Create a dictionary mapping the feature name to the 
    # tf.Example-compatible data type.
    feature = {
        'image': _bytes_feature(tf.io.encode_png(image), is_list=False),
        'image_name': _bytes_feature(image_name, is_list=False),
        'target': _bytes_feature(target, is_list=False),
    }

    # Create a Features message using tf.train.Example.
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

def write_tfrecords(ds_np, n_recs=64, n_ex_per_rec=341, out_dir="/kaggle/working/train_records"):
    if not os.path.isdir(out_dir):
        os.makedirs(out_dir, exist_ok=True)
    for i in tqdm(range(n_recs), total=n_recs):
        print(f"\n... Writing TFRecord {i+1} of {n_recs} ...\n")
        tfrec_path = os.path.join(out_dir, f"{out_dir.rsplit('_', 1)[1]}_{(i+1):02}_{n_recs:02}.tfrec")
        with tf.io.TFRecordWriter(tfrec_path) as writer:
            for j in tqdm(range(n_ex_per_rec), total=n_ex_per_rec):
                try:
                    example = serialize(*next(ds_np))
                    writer.write(example)
                except:
                    break

In [ ]:
N_TRAIN_RECS = 64
N_EX = len(train_df)
N_EX_PER_REC = int(np.ceil(N_EX/64))
write_tfrecords(train_ds.as_numpy_iterator(), out_dir="/kaggle/working/train_slide_records")

In [ ]:
NEW_TRAIN_TFREC_PATHS = [
    os.path.join("/kaggle/working/train_slide_records", f_name) \
    for f_name in os.listdir("/kaggle/working/train_slide_records")
]
new_raw_train_ds = tf.data.TFRecordDataset(NEW_TRAIN_TFREC_PATHS, num_parallel_reads=None)
new_train_ds = new_raw_train_ds.map(lambda x: decode(x, n_channels=3))

# See examples
print("\n ... NOTICE THE IMAGES ARE CLUMPED TOGETHER BY CHANNEL FOR A GIVEN ID ...\n")
for i, (img, image_name, lbl) in enumerate(new_train_ds.take(3)):
    print(f"IMAGE SHAPE : {img.shape}")
    print(f"IMG #{i//4} -- IMAGE NAME  : {image_name.numpy().decode()}")
    print(f"IMAGE LABEL : {lbl}\n")
    plt.imshow(img.numpy().astype(np.uint8))
    plt.show()